In [1]:
from experiments.regression_0.analysis import Readers
from ga_regression import *

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
file = Path(r"D:\resynth\run_48_49\with_dirac_eigs\2025-07-24-07-07-01\opts_and_metadata.pt") # dirac eigs
readers = Readers.from_file(file)

In [15]:
r0 = readers[0]
m0 = r0._meta
expt0 = m0.experiment(train_dataset=r0.train_scenes, test_dataset=r0.test_scenes)
dataset0, _s, _r = m0.load_dataset(weights=None)
d0 = dataset0[0]

In [16]:
r1 = readers[2]
m1 = r1._meta
expt1 = m1.experiment(train_dataset=r1.train_scenes, test_dataset=r1.test_scenes)
dataset1, _s, _r = m1.load_dataset(weights=None)
d1 = dataset1[0]

In [19]:
d0.evecs

tensor([[-2.3657, -2.8997, -3.5221,  ..., -1.8561, -0.1609,  1.1776],
        [-2.3657,  3.9496, -1.1058,  ...,  1.1162, -5.7006,  0.5749],
        [-2.3657,  1.4467,  1.9699,  ...,  2.3772, -0.9227,  0.8147],
        ...,
        [-2.3657, -3.9168, -3.7006,  ...,  0.2445, -0.0057, -0.0116],
        [-2.3657,  2.4288,  2.0619,  ...,  0.6390, -0.1948,  0.0199],
        [-2.3657,  1.7122,  2.6484,  ..., -0.4518, -0.3237, -0.2428]])

In [26]:
self = expt1
data = d1

verts = data.verts.to(self.device)
faces = data.faces.to(self.device)
# _frames = frames.to(self.device)
mass = data.mass.to(self.device)
L = data.L.to(self.device)
evals = data.evals.to(self.device)
evecs = data.evecs.to(self.device)
gradX = data.gradX.to(self.device)
gradY = data.gradY.to(self.device)

if (weights := data.weight) is not None:
    if not isinstance(weights, torch.Tensor):
        weights = torch.Tensor(np.asarray(weights))
    weights = weights.to(self.device)

if not isinstance((labels := data.labels), torch.Tensor):
    labels = torch.Tensor(labels)

labels = labels.to(self.device)

# if augment_random_rotate:
#     verts = diffusion_net.utils.random_rotate_points(verts)

# Construct features
match self.metadata.input_features:
    case 'xyz':
        features = verts
    case 'hks' | ('dirac', _):
        features = diffusion_net.geometry.compute_hks_autoscale(evals, evecs, 16)
    case x:
        raise NotImplementedError(x)

In [32]:
evals.clamp(min=0)

tensor([1.4047e-01, 1.2470e+02, 0.0000e+00, 1.7497e+02, 2.1494e+02, 0.0000e+00,
        0.0000e+00, 2.5141e+02, 3.2346e+02, 0.0000e+00, 3.5712e+02, 0.0000e+00,
        3.8952e+02, 0.0000e+00, 4.1862e+02, 0.0000e+00, 4.8330e+02, 0.0000e+00,
        5.1190e+02, 0.0000e+00, 5.5946e+02, 0.0000e+00, 6.2478e+02, 0.0000e+00,
        6.5338e+02, 6.8160e+02, 7.2002e+02, 0.0000e+00, 0.0000e+00, 7.7261e+02,
        0.0000e+00, 0.0000e+00, 8.3865e+02, 8.8123e+02, 0.0000e+00, 9.3330e+02,
        0.0000e+00, 9.9684e+02, 1.0442e+03, 0.0000e+00, 1.0802e+03, 0.0000e+00,
        1.1288e+03, 0.0000e+00, 1.1862e+03, 1.2245e+03, 0.0000e+00, 1.2785e+03,
        0.0000e+00, 0.0000e+00, 1.3597e+03, 1.4108e+03, 0.0000e+00, 1.4920e+03,
        0.0000e+00, 1.5430e+03, 0.0000e+00, 1.5978e+03, 0.0000e+00, 1.6757e+03,
        0.0000e+00, 0.0000e+00, 1.7719e+03, 1.8315e+03, 0.0000e+00, 0.0000e+00,
        1.9429e+03, 0.0000e+00, 1.9942e+03, 0.0000e+00, 2.0803e+03, 0.0000e+00,
        2.2161e+03, 0.0000e+00, 2.2742e+

In [31]:
diffusion_net.geometry.compute_hks_autoscale(evals.abs(), evecs, 16).isinf().any()

tensor(False, device='cuda:0')